## 📄 Case Study: Estimating Patient Waiting Times in Operating Rooms

### Background

A hospital operates a **single operating room**, where patients are scheduled and treated on a **first-come, first-served** basis. Over the past year, the hospital has collected data on interarrival times and surgery durations, which is available in an Excel file.

### Objective

Using the historical data, the hospital aims to estimate the **average waiting time** that patients may experience in the **upcoming  month**. The results will help improve operating room scheduling and patient flow efficiency.onth**.


**First load the excel data to jupyternotebook**

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats

# Load your Excel data to df
df = pd.read_excel("Operating room_2024.xlsx")

# convert the date column into correct date format
df['Date'] = pd.to_datetime(df['Date'])


**Explore the data to detect possible features.** You can explore freely in different ways.

In [ ]:
# ---------------------------
# group data according to the date column, all the data within the same day are aggregated by mean, for both interarrival time and service time.
# Rename the two columns as mean interarrival and mean service
# ---------------------------
daily = df.groupby('Date').agg({
    'Interarrival Time (hrs)': 'mean',
    'Service Time (hrs)': 'mean'
}).rename(columns={
    'Interarrival Time (hrs)': 'Mean Interarrival',
    'Service Time (hrs)': 'Mean Service'
})

# 2-day moving averages
daily['Interarrival MA'] = daily['Mean Interarrival'].rolling(window=2).mean()
daily['Service MA'] = daily['Mean Service'].rolling(window=2).mean()

# Plot moving averages
plt.figure(figsize=(14, 6))
plt.subplot(2, 1, 1)
plt.plot(daily['Mean Interarrival'], label='Daily Mean', alpha=0.5)
plt.plot(daily['Interarrival MA'], label='2-day MA', color='blue')
plt.title("Moving Average of Interarrival Time")
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(daily['Mean Service'], label='Daily Mean', alpha=0.5)
plt.plot(daily['Service MA'], label='2-day MA', color='green')
plt.title("Moving Average of Service Time")
plt.legend()
plt.tight_layout()
plt.show()

# ---------------------------
# 2. Periodicity: Day of Week Patterns
#  cagetories data by day, print 
# ---------------------------
day_avg = df.groupby('Day').agg({
    'Interarrival Time (hrs)': 'mean',
    'Service Time (hrs)': 'mean'
})

# Define the correct weekday order
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Reorder the DataFrame
day_avg = day_avg.reindex(weekday_order)

# Plot averages by day in bar chart
day_avg.plot(kind='bar', figsize=(10, 5))
plt.title("Average Interarrival and Service Time by Day of Week")
plt.ylabel("Time (hours)")
plt.grid(axis='y')
plt.tight_layout()
plt.show()

**The figures above do not show clear pattern per season. However, we can see the mean service times vary per day within a week, the differences of mean interarrival times are less clear. To better study the differences within each week, we create boxplot for each day for both interarrival times and service times.**

In [ ]:
# Plot boxplot for interarrival time by day
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Day', y='Interarrival Time (hrs)', order=[
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])
plt.title("Boxplot of Interarrival Time by Day of the Week")
plt.ylabel("Interarrival Time (hours)")
plt.xlabel("Day of the Week")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

# Plot boxplot of service time by day
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Day', y='Service Time (hrs)', order=[
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])
plt.title("Boxplot of Service Time by Day of the Week")
plt.ylabel("Service Time (hours)")
plt.xlabel("Day of the Week")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

**From the above figure, we can see that interarrival time does not change much, service times are different per day. We can safely make the following assumptions: interarrival times follow the same distribution per day; service time distributions differ day to day. Next we look for the right distribution to fit the interarrival times and service times, also estimate the associated parameters.**



**We first create a histogram to visualize the distributions of interarrival times**

In [ ]:
# ---------------------------
# 3. Distribution Fitting: Interarrival Time
# ---------------------------
interarrival = df['Interarrival Time (hrs)']

# Step 1: Visualize the empirical distribution
plt.figure(figsize=(6, 4))
sns.histplot(interarrival, bins=50, kde=True)
plt.title("Histogram of Interarrival Times")
plt.xlabel("Interarrival Time (hrs)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


**It looks like lognormal distribution. The next step is to use Q-Q plot to visually check the fit of lognormal distribution (I also checked the fit of exponential distribution, to show you the differences), and use k-s test to formally test the fit of lognormal distribution. We use MLE to estimate the parameters.**

In [ ]:
# Parameters for lognormal distribution (fit to data) using MLE
shape, loc, scale = stats.lognorm.fit(interarrival, floc=0)  # Fix loc=0 for interarrival times

# QQ plot for lognormal
plt.figure(figsize=(12,5))

# plot the data "interarrival" and the estimated lognormal distribution
plt.subplot(1,2,1)
stats.probplot(interarrival, dist=stats.lognorm(s=shape, loc=loc, scale=scale), plot=plt)
plt.title('QQ Plot - Lognormal')

# K-S test for lognormal
D_lognorm, p_lognorm = stats.kstest(interarrival, 'lognorm', args=(shape, loc, scale))


# ----------- 2. Exponential QQ Plot and K-S Test -----------

# Fit exponential distribution (loc fixed at 0 since interarrival times >= 0)
loc_exp, scale_exp = 0, np.mean(interarrival)  # MLE for exponential mean = scale parameter

# QQ plot for exponential
plt.subplot(1,2,2)
stats.probplot(interarrival, dist=stats.expon, sparams=(loc_exp, scale_exp), plot=plt)
plt.title('QQ Plot - Exponential')

# K-S test for exponential
D_exp, p_exp = stats.kstest(interarrival, 'expon', args=(loc_exp, scale_exp))


plt.tight_layout()
plt.show()

# Print K-S test results
print(f'Lognormal K-S test: D = {D_lognorm:.4f}, p-value = {p_lognorm:.4f}')
print(f'Exponential K-S test: D = {D_exp:.4f}, p-value = {p_exp:.4f}')

In [23]:
print(f"Lognormal fitted parameters for interarrival time: shape (sigma) = {shape:.4f}, loc = {loc:.4f}, scale (exp(mu)) = {scale:.4f}")


Lognormal fitted parameters for interarrival time: shape (sigma) = 0.4642, loc = 0.0000, scale (exp(mu)) = 2.0348


**Next we apply the same steps to estimate the parameters for service times.**

In [ ]:
# Ensure days appear in standard order
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Set up plot: 2 rows (histogram + QQ) per day in a week
plt.figure(figsize=(14, 20))
plot_idx = 1

for day in weekday_order:
    # extract 'Service Time (hrs)' only from rows with df['Day'] = day; i.e., extract service times for mondays; tuesdays;...
    service_times = df[df['Day'] == day]['Service Time (hrs)']

    if len(service_times) < 10:
        print(f"{day}: Not enough data for analysis.")
        continue

    # Fit exponential distribution using MLE
    loc, scale = 0, np.mean(service_times)
    
    # Histogram
    plt.subplot(7, 2, plot_idx)
    sns.histplot(service_times, stat='density', bins=30, color='skyblue', edgecolor='black')
    x = np.linspace(0, service_times.max(), 100)
    y = stats.expon.pdf(x, loc=loc, scale=scale)
    plt.plot(x, y, 'r--', label='Fitted Exponential PDF')
    plt.title(f'Histogram - {day}')
    plt.legend()
    plot_idx += 1

    # QQ Plot
    plt.subplot(7, 2, plot_idx)
    stats.probplot(service_times, dist=stats.expon, sparams=(loc, scale), plot=plt)
    plt.title(f'QQ Plot - {day}')
    plot_idx += 1

    # K-S test
    D, p = stats.kstest(service_times, 'expon', args=(loc, scale))

    # Print results
    print(f"{day}: loc = {loc:.4f}, scale (mean) = {scale:.4f}, K-S D = {D:.4f}, p-value = {p:.4f}")

plt.tight_layout()
plt.show()

**Now the arrival process and service time distributions are all clear and summarised as follows: 
Interarrival time is lognormally distributed: shape (sigma) = 0.4642, scale (exp(mu)) = 4.4140;** 


**service time exponetially distributed:
Monday: scale (mean) = 1.9142; 
Tuesday:scale (mean) = 5.9261; 
Wednesday: scale (mean) = 2.6653; 
Thursday: scale (mean) = 0.9751; 
Friday: scale (mean) = 5.9415; 
Saturday: scale (mean) = 0.9453; 
Sunday: scale (mean) = 2.0348**

**Next we build a DES model for the G/G/1 queue**

We will first create a single sample path for the coming week, including the arrivals and departures. We estimate the average waiting of all the patients, denoted by $W_1$; (this is created by Discrete Event Simulation)

Then we replicate step 1 for a large number of times, say 3000, the average waiting times of each sample path is denoted by $W_1$, $W_2$, $\ldots$, $W_{3000}$. 

Lastly, using Monte Carlo Simulation. We use $\bar{W}=\frac{\sum_{j=1}^{3000}W_j}{3000}$ to approximate the average waiting time of the patients arriving in the coming week, also report the confidence interval of $\bar{W}$.

In the DES model, the system state is ($n$), representing the number of patients in the system, including the patients waiting in the queue and in service. Events lead to system state changes are arrivals $\alpha$ and departures $\beta$. The interarrival times and service times have been estimated in the first few steps. The state transitions can be described below:
$\Phi((n),\alpha)=(n+1)$, $\Phi((n),\beta)=(n-1)$.

In [12]:
# Simulation parameters
SIMULATION_DAYS = 7
HOURS_PER_DAY = 24
SIMULATION_TIME = SIMULATION_DAYS * HOURS_PER_DAY  # total time in hours

# Interarrival time parameters (stationary lognormal)
interarrival_loc = 0
interarrival_scale = 4.4140
interarrival_shape = 0.4642

# Service time parameters (exponential) per weekday
service_params = {
    'Monday':    {'loc': 0, 'scale': 1.9142},
    'Tuesday':   {'loc': 0, 'scale': 5.9261},
    'Wednesday': {'loc': 0, 'scale': 2.6653},
    'Thursday':  {'loc': 0, 'scale':  0.9751},
    'Friday':    {'loc': 0, 'scale': 5.9415},
    'Saturday':  {'loc': 0, 'scale': 0.9453},
    'Sunday':    {'loc': 0, 'scale': 2.0348},
}

weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def run_simulation():
    current_time = 0
    waiting_times = []
    next_available_time = 0

    while current_time < SIMULATION_TIME:  # stop the simulation at the end of the last day
        day_index = int((current_time // HOURS_PER_DAY))
        weekday = weekdays[day_index]

        # Interarrival time (lognormal, stationary)
        interarrival = stats.lognorm.rvs(s=interarrival_shape, loc=interarrival_loc, scale=interarrival_scale)

        current_time += interarrival
        if current_time >= SIMULATION_TIME:
            break

        # Service time (exponential) per weekday
        s_params = service_params[weekday]
        service_time = stats.expon.rvs(loc=s_params['loc'], scale=s_params['scale'])

        # Compute waiting time
        wait_time = max(next_available_time - current_time, 0)
        waiting_times.append(wait_time)
        next_available_time = max(current_time, next_available_time) + service_time

    return np.mean(waiting_times) if waiting_times else 0

run_simulation()

0.6228052911014053

**How to validate the code? there are multiple ways: adjust the parameters to check whether it is consistent with intuition. This is ignored in the code**

**Next we replicate the DES simulation model for 3000 times.**

In [25]:
from scipy.stats import norm
def replicate_simulation(num_replications=3000, random_seed=42): 
    np.random.seed(random_seed)
    avg_wait_times = [run_simulation() for _ in range(num_replications)]

    sample_mean = np.mean(avg_wait_times)
    sample_std = np.std(avg_wait_times, ddof=1)
    conf_level = 0.95
    alpha = 1 - conf_level  # sufficient level
    
    # Use normal distribution's critical value (z-critical)
    z_crit = norm.ppf(1 - alpha/2)

    margin_of_error = z_crit * (sample_std / np.sqrt(num_replications))
    conf_interval = (sample_mean - margin_of_error, sample_mean + margin_of_error)

    print(f"Sample mean average waiting time over {num_replications} replications: {sample_mean:.4f} hours")
    print(f"95% confidence interval: ({conf_interval[0]:.4f}, {conf_interval[1]:.4f}) hours")
    print(f"Sample mean average waiting time in minutes: {sample_mean * 60:.2f} minutes")
    print(f"95% confidence interval in minutes: ({conf_interval[0] * 60:.2f}, {conf_interval[1] * 60:.2f}) minutes")

# Run the replication and report results
replicate_simulation()


Sample mean average waiting time over 3000 replications: 4.3372 hours
95% confidence interval: (4.1706, 4.5038) hours
Sample mean average waiting time in minutes: 260.23 minutes
95% confidence interval in minutes: (250.24, 270.23) minutes
